In [1]:
import sys
import os

sys.path.insert(0, os.path.abspath("."))
sys.path.append(os.path.abspath("../../"))

In [2]:
from desc.backend import jnp, jit
from desc.grid import ConcentricGrid
from desc.equilibrium import Equilibrium
from desc.geometry import FourierRZToroidalSurface
from desc.profiles import PowerSeriesProfile
from desc.compute import data_index
from desc.compute._core import (
    compute_rotational_transform,
    compute_rotational_transform_v2,
)
from desc.transform import Transform
import numpy as np

DESC version 0.5.0+43.g57e5849.dirty, using JAX backend, jax version=0.2.25, jaxlib version=0.1.76, dtype=float64
Using device: CPU, with 10.88 GB available memory


In [3]:
jnp.set_printoptions(precision=3, floatmode="fixed")
rng = np.random.default_rng()

In [4]:
L = rng.integers(low=1, high=100)
M = rng.integers(low=1, high=100)
N = rng.integers(low=1, high=100)
print(L, M, N)
grid = ConcentricGrid(L=L, N=N, M=M, node_pattern="jacobi")
# print("nodes", "             ", "spacing")
# for a, b in zip(grid.nodes, grid.spacing):
#     print(a, b)

74 25 28


## Bulk flux surface averaging test
The tests pass. As shown by timeit the _surface_sums no loop algorithm is much faster.

In [5]:
@jit
def _surface_sums(surf_label, unique_append_upperbound, weights):
    """
    Parameters
    ----------
    surf_label : ndarray
        The surface label. Elements of a coordinate in the collocation grid.
        i.e. grid.nodes[:, 0]
    unique_append_upperbound : ndarray
        Sorted unique elements of surf_label with the upper bound of that coordinate appended.
        i.e. grid.unique_rho + [1]
    weights : ndarray
        Node at surf_label[i]'s contribution to its surface's sum is weights[i].
        For an integral, this could be: ds * function_to_integrate.

    Returns
    -------
    ndarray
        An array of weighted sums over each surface.
        The returned array has length = len(unique_append_upperbound) - 1.
    """
    # DESIRED ALGORITHM
    # collect collocation node indices for each rho surface
    # surfaces = dict()
    # for index, rho in enumerate(surf_label):
    #     surfaces.setdefault(rho, list()).append(index)
    # integration over non-contiguous elements
    # for i, surface in enumerate(surfaces.values()):
    #     surface_sums[i] = weights[surface].sum()

    # NO LOOP IMPLEMENTATION
    # Separate collocation nodes into bins with boundaries at unique values of rho.
    # This groups nodes with identical rho values.
    # Each is assigned a weight of their contribution to the integral.
    # The elements of each bin are summed, performing the integration.
    return jnp.histogram(surf_label, bins=unique_append_upperbound, weights=weights)[0]

In [6]:
rho = grid.nodes[:, 0]
weights = np.random.random_sample(size=len(rho))

In [7]:
%%timeit

iota_1 = np.zeros(grid.num_rho)
# DESIRED ALGORITHM
# collect collocation node indices for each rho surface
surfaces = dict()
for index, r in enumerate(rho):
    surfaces.setdefault(r, list()).append(index)
# integration over non-contiguous elements
for i, surface in enumerate(surfaces.values()):
    iota_1[i] = weights[surface].sum()

15.8 ms ± 307 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [8]:
%%timeit

# NO LOOP IMPLEMENTATION
bins = jnp.append(grid.unique_rho, 1)
iota_2 = _surface_sums(rho, bins, weights)
# bincount, bins = jnp.histogram(rho, bins=bins)
# print(grid.unique_rho)
# print(bincount)
# print(bins)

1.49 ms ± 15 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [9]:
# must remove %%timeit to test
# print(iota_1)
# print(iota_2)
# assert jnp.allclose(iota_1, iota_2)

In [10]:
# no longer used in code
# custom implementation of jnp.unique(), fails jit
# @jit
def get_unique_rho(grid):
    change_zeta = jnp.where(
        jnp.diff(grid.nodes[:, 2], prepend=jnp.nan), size=grid.num_zeta
    )[0]
    stop_zeta = change_zeta[1] if grid.num_zeta > 1 else None
    change_rho = jnp.where(
        jnp.diff(grid.nodes[:stop_zeta, 0], prepend=jnp.nan), size=grid.num_rho
    )[0]
    return grid.nodes[change_rho, 0]

assert jnp.allclose(grid.unique_rho, get_unique_rho(grid))

## Axisymmetric, vacuum, no current test
Want to test to see if the returned rotational transform profile is 0 when toroidal current input is 0.
This should be a good test because the bulk of the computation lies on enforcing the zero
toroidal current algorithm for the geometry of the device. And when a non-zero toroidal current is specified we just add it to the numerator because that is the delta poloidal flux term.

In [11]:
eq = Equilibrium()
print(eq.Psi)
print(eq.i_l)
compute_rotational_transform(eq.i_l, eq.iota)

1.0
[0.000]


{'iota': DeviceArray([], dtype=float64),
 'iota_r': DeviceArray([], dtype=float64),
 'iota_rr': DeviceArray([], dtype=float64)}

In [12]:
# is this the canoncial way to request multiple derivatives be computed in a transform?
r_derivs = np.stack(
    (
        data_index["g_tt"]["R_derivs"][0],
        data_index["e_theta_r"]["R_derivs"][0],
        data_index["e_theta_rr"]["R_derivs"][0],
        data_index["g_tz"]["R_derivs"][0],
        data_index["g_tz"]["R_derivs"][1],
        data_index["g_tz"]["R_derivs"][2],
        data_index["e_zeta_r"]["R_derivs"][0],
        data_index["e_zeta_r"]["R_derivs"][1],
        data_index["e_zeta_rr"]["R_derivs"][0],
        data_index["e_zeta_rr"]["R_derivs"][1],
        data_index["R_rrz"]["R_derivs"][0],
    )
)
l_derivs = np.stack(
    (
        data_index["lambda_t"]["L_derivs"][0],
        data_index["lambda_rt"]["L_derivs"][0],
        data_index["lambda_rrt"]["L_derivs"][0],
        data_index["lambda_z"]["L_derivs"][0],
        data_index["lambda_rz"]["L_derivs"][0],
        data_index["lambda_rrz"]["L_derivs"][0],
    )
)
R_transform = Transform(grid, eq.R_basis, derivs=r_derivs, build=True)
Z_transform = Transform(grid, eq.Z_basis, derivs=r_derivs, build=True)
L_transform = Transform(grid, eq.L_basis, derivs=l_derivs, build=True)

# The * operation between psi_r and the term in the following parenthesis fails
# The error clams they are different sizes, but I'm not sure why.
compute_rotational_transform_v2(
    eq.R_lmn,
    eq.Z_lmn,
    eq.L_lmn,
    R_transform,
    Z_transform,
    L_transform,
    eq.Psi,
    I_l=np.zeros(2),
    toroidal_current=PowerSeriesProfile(params=[1, 1.5], modes=[0, 2]),
)

TypeError: mul got incompatible shapes for broadcasting: (55974,), (0,).